### Dummy Train to Overfit

In [1]:
from dataset.dataset import *
from dataset.transform import *
from model.nets import UNet
from model.loss import dice_loss

from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [2]:
path = '../data/ds_bowl_2018/full_data/'

In [8]:
def augment(image, mask, index, sz = 256):
    image, mask = fix_resize_transform2(image, mask, sz, sz)
    image = (image.transpose((2,1,0))/255) #normalize, batchnorm will be better for future
    mask = mask[None, :]
    return image, mask, index

In [9]:
nuclei_ds = NucleiDataset(path, transform=augment, mode='train')

In [10]:
nuclei_dl = DataLoader(nuclei_ds, batch_size=2, shuffle=True)

In [11]:
inputs = next(iter(nuclei_dl))

In [14]:
net = UNet(1, 3).double()
optimizer = optim.Adam(net.parameters())

In [15]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for data, target, *_ in inputs:
        
        # wrap them in Variable
        data, target = Variable(data.double()), Variable(target.double())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        logits = net(data)
        outputs = F.sigmoid(logits)
        loss = dice_loss(outputs, target)
        loss.backward()
        optimizer.step()
        print(loss)
print('Finished Training')

ValueError: Expected 4D tensor as input, got 3D tensor instead.